### 連接kaggle資料

In [ ]:
! pip install -q kaggle
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 7.9 MB/s eta 0:00:00


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'nthu-mlfinal-project'

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
 83% 121M/145M [00:00<00:00, 192MB/s] 
100% 145M/145M [00:00<00:00, 212MB/s]


In [ ]:
!unzip nthu-mlfinal-project

Archive:  nthu-mlfinal-project.zip
  inflating: light_test_source_labels.csv  
  inflating: light_train_source_labels.csv  
  inflating: light_train_target_labels.csv  
  inflating: sample.csv              
  inflating: test_source_events.csv  
  inflating: train_source_events.csv  
  inflating: train_target_events.csv  


package

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import datetime
from sklearn.model_selection import GridSearchCV

## Feature Engineering

In [ ]:
train_source = pd.read_csv('train_source_events.csv')
train_source.shape

(9706783, 10)

In [ ]:
tid = list(train_source['title_id'])
drama = set(tid)
count = dict()
for i in drama:
  count[i] = 0
for i in tid:
  count[i]+=1

In [ ]:
c_sort = sorted(count.items(), key=lambda x:x[1], reverse = True)
#c_sort
#68, 14, 93

加上星期幾及所屬timeslot(0,1,2,3)

In [ ]:
def week(date):
  week_list = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
  struct_time = time.localtime(date)
  w_day = struct_time.tm_wday
  hour = struct_time.tm_hour
  if hour < 1:
    w_day = w_day - 1 if w_day !=0 else 6
  return week_list[w_day]

def timeslot(date):
  struct_time = time.localtime(date)
  hour = struct_time.tm_hour
  if hour < 1:
    time_slot = 3
  elif hour < 9 :
    time_slot = 0
  elif hour < 17:
    time_slot = 1
  elif hour < 21:
    time_slot = 2
  else:
    time_slot = 3
  return time_slot

In [ ]:
train_source['weekday'] = train_source['event_time'].apply(week)
train_source['timeslot'] = train_source['event_time'].apply(timeslot)

In [ ]:
timeString = "2022-08-20 1:00:00" # 時間格式為字串
struct_time = time.strptime(timeString, "%Y-%m-%d %H:%M:%S") # 轉成時間元組
week1 = int(time.mktime(struct_time)) # 轉成時間戳

timeString = "2022-08-13 1:00:00" # 時間格式為字串
struct_time = time.strptime(timeString, "%Y-%m-%d %H:%M:%S") # 轉成時間元組
week2 = int(time.mktime(struct_time))

timeString = "2022-08-6 1:00:00" # 時間格式為字串
struct_time = time.strptime(timeString, "%Y-%m-%d %H:%M:%S") # 轉成時間元組
week3 = int(time.mktime(struct_time))

In [ ]:
train_source

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,title_in_simulcast,internet_connection_type,weekday,timeslot
0,0,220,476,138,1.639215e+09,405,0,0,1,5,Saturday,1
1,0,220,476,212,1.639215e+09,30,1,0,1,5,Saturday,1
2,0,220,476,212,1.639215e+09,59,1,0,1,5,Saturday,1
3,0,220,476,212,1.639215e+09,54,1,0,1,5,Saturday,1
4,0,220,476,212,1.639215e+09,11,0,0,1,5,Saturday,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9706778,30459,139931,4235187,11,1.659684e+09,823,0,2,0,4,Friday,0
9706779,30459,140428,4231782,259,1.659752e+09,2030,2,2,1,4,Saturday,0
9706780,30459,140428,4231782,259,1.659752e+09,56,0,2,1,4,Saturday,0
9706781,30459,140470,4235269,259,1.659951e+09,513,0,2,1,4,Monday,1


In [ ]:
train_source.loc[train_source['event_time'] > week1  , 'played_duration'] *= 4
train_source.loc[train_source['event_time'] > week2, 'played_duration'] *= 3
train_source.loc[train_source['event_time'] > week3, 'played_duration'] *= 2
train_source.loc[train_source['event_time'] < week3, 'played_duration'] *= 0.5

In [ ]:
train_source

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,title_in_simulcast,internet_connection_type,weekday,timeslot
0,0,220,476,138,1.639215e+09,202.5,0,0,1,5,Saturday,1
1,0,220,476,212,1.639215e+09,15.0,1,0,1,5,Saturday,1
2,0,220,476,212,1.639215e+09,29.5,1,0,1,5,Saturday,1
3,0,220,476,212,1.639215e+09,27.0,1,0,1,5,Saturday,1
4,0,220,476,212,1.639215e+09,5.5,0,0,1,5,Saturday,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9706778,30459,139931,4235187,11,1.659684e+09,411.5,0,2,0,4,Friday,0
9706779,30459,140428,4231782,259,1.659752e+09,4060.0,2,2,1,4,Saturday,0
9706780,30459,140428,4231782,259,1.659752e+09,112.0,0,2,1,4,Saturday,0
9706781,30459,140470,4235269,259,1.659951e+09,1026.0,0,2,1,4,Monday,1


### 熱播劇


In [ ]:
hot_drama = dict()

for i in range(28):
  hot_drama[i] = dict()
  for j in range(30460):
    hot_drama[i][j] = []

for i in range(len(train_source['user_id'])):
  event = train_source['event_time'][i]
  id = train_source['user_id'][i]
  simul = train_source['title_id'][i]
  if(event < 1659056400):
    continue
  if(event >= 1660957200 and event < 1660986000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)
  elif(event >= 1660352400 and event < 1660381200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)
  elif(event >= 1659747600 and event < 1659776400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)
  elif(event >= 1659142800 and event < 1659171600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)

  if(event >= 1660986000 and event < 1661014800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)
  elif(event >= 1660381200 and event < 1660410000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)
  elif(event >= 1659776400 and event < 1659805200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)
  elif(event >= 1659171600 and event < 1659200400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)

  if(event >= 1661014800 and event < 1661029200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)
  elif(event >= 1660410000 and event < 1660424400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)
  elif(event >= 1659805200 and event < 1659819600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)
  elif(event >= 1659200400 and event < 1659214800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)

  if(event >= 1661029200 and event < 1661043600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)
  elif(event >= 1660424400 and event < 1660438800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)
  elif(event >= 1659819600 and event < 1659834000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)
  elif(event >= 1659214800 and event < 1659229200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)

  if(event >= 1661043600 and event < 1661072400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)
  elif(event >= 1660438800 and event < 1660467600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)
  elif(event >= 1659834000 and event < 1659862800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)
  elif(event >= 1659229200 and event < 1659258000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)

  if(event >= 1661072400 and event < 1661101200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)
  elif(event >= 1660467600 and event < 1660496400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)
  elif(event >= 1659862800 and event < 1659891600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)
  elif(event >= 1659258000 and event < 1659286800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)

  if(event >= 1661101200 and event < 1661115600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)
  elif(event >= 1660496400 and event < 1660510800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)
  elif(event >= 1659891600 and event < 1659906000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)
  elif(event >= 1659286800 and event < 1659301200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)

  if(event >= 1661115600 and event < 1661130000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)
  elif(event >= 1660510800 and event < 1660525200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)
  elif(event >= 1659906000 and event < 1659920400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)
  elif(event >= 1659301200 and event < 1659315600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)

  if(event >= 1661130000 and event < 1661158800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)
  elif(event >= 1660525200 and event < 1660554000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)
  elif(event >= 1659920400 and event < 1659949200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)
  elif(event >= 1659315600 and event < 1659344400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)

  if(event >= 1661158800 and event < 1661187600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)
  elif(event >= 1660554000 and event < 1660582800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)
  elif(event >= 1659949200 and event < 1659978000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)
  elif(event >= 1659344400 and event < 1659373200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)

  if(event >= 1661187600 and event < 1661202000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)
  elif(event >= 1660582800 and event < 1660597200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)
  elif(event >= 1659978000 and event < 1659992400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)
  elif(event >= 1659373200 and event < 1659387600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)

  if(event >= 1661202000 and event < 1661216400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)
  elif(event >= 1660597200 and event < 1660611600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)
  elif(event >= 1659992400 and event < 1660006800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)
  elif(event >= 1659387600 and event < 1659402000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)

  if(event >= 1661216400 and event < 1661245200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)
  elif(event >= 1660611600 and event < 1660640400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)
  elif(event >= 1660006800 and event < 1660035600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)
  elif(event >= 1659402000 and event < 1659430800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)

  if(event >= 1661245200 and event < 1661274000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)
  elif(event >= 1660640400 and event < 1660669200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)
  elif(event >= 1660035600 and event < 1660064400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)
  elif(event >= 1659430800 and event < 1659459600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)

  if(event >= 1661274000 and event < 1661288400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)
  elif(event >= 1660669200 and event < 1660683600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)
  elif(event >= 1660064400 and event < 1660078800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)
  elif(event >= 1659459600 and event < 1659474000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)

  if(event >= 1661288400 and event < 1661302800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)
  elif(event >= 1660683600 and event < 1660698000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)
  elif(event >= 1660078800 and event < 1660093200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)
  elif(event >= 1659474000 and event < 1659488400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)

  if(event >= 1661302800 and event < 1661331600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)
  elif(event >= 1660698000 and event < 1660726800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)
  elif(event >= 1660093200 and event < 1660122000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)
  elif(event >= 1659488400 and event < 1659517200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)

  if(event >= 1661331600 and event < 1661360400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)
  elif(event >= 1660726800 and event < 1660755600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)
  elif(event >= 1660122000 and event < 1660150800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)
  elif(event >= 1659517200 and event < 1659546000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)

  if(event >= 1661360400 and event < 1661374800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)
  elif(event >= 1660755600 and event < 1660770000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)
  elif(event >= 1660150800 and event < 1660165200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)
  elif(event >= 1659546000 and event < 1659560400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)

  if(event >= 1661374800 and event < 1661389200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)
  elif(event >= 1660770000 and event < 1660784400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)
  elif(event >= 1660165200 and event < 1660179600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)
  elif(event >= 1659560400 and event < 1659574800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)

  if(event >= 1661389200 and event < 1661418000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)
  elif(event >= 1660784400 and event < 1660813200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)
  elif(event >= 1660179600 and event < 1660208400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)
  elif(event >= 1659574800 and event < 1659603600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)

  if(event >= 1661418000 and event < 1661446800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)
  elif(event >= 1660813200 and event < 1660842000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)
  elif(event >= 1660208400 and event < 1660237200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)
  elif(event >= 1659603600 and event < 1659632400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)

  if(event >= 1661446800 and event < 1661461200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)
  elif(event >= 1660842000 and event < 1660856400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)
  elif(event >= 1660237200 and event < 1660251600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)
  elif(event >= 1659632400 and event < 1659646800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)

  if(event >= 1661461200 and event < 1661475600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)
  elif(event >= 1660856400 and event < 1660870800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)
  elif(event >= 1660251600 and event < 1660266000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)
  elif(event >= 1659646800 and event < 1659661200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)

  if(event >= 1661475600 and event < 1661504400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)
  elif(event >= 1660870800 and event < 1660899600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)
  elif(event >= 1660266000 and event < 1660294800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)
  elif(event >= 1659661200 and event < 1659690000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)

  if(event >= 1661504400 and event < 1661533200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)
  elif(event >= 1660899600 and event < 1660928400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)
  elif(event >= 1660294800 and event < 1660323600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)
  elif(event >= 1659690000 and event < 1659718800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)

  if(event >= 1661533200 and event < 1661547600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)
  elif(event >= 1660928400 and event < 1660942800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)
  elif(event >= 1660323600 and event < 1660338000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)
  elif(event >= 1659718800 and event < 1659733200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)

  if(event >= 1661547600 and event < 1661562000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)
  elif(event >= 1660942800 and event < 1660957200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)
  elif(event >= 1660338000 and event < 1660352400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)
  elif(event >= 1659733200 and event < 1659747600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)

In [ ]:
hot_drate = []
for i in range(28):
  for j in range(30460):
    if(len(hot_drama[i][j]) == 0):
      hot_drate.append(0)
    else:
      hot_drate.append(sum(hot_drama[i][j]) / len(hot_drama[i][j]))
      #hot_rate.append(sum(hot_title[i][j]))

In [ ]:
hot_title = dict()
for i in range(28):
  hot_title[i] = dict()
  for j in range(30460):
    hot_title[i][j] = []
for i in range(len(train_source['user_id'])):
  event = train_source['event_time'][i]
  id = train_source['user_id'][i]
  simul = train_source['title_in_simulcast'][i]
  if(event < 1659056400):
    continue
  if(event >= 1660957200 and event < 1660986000):
    hot_title[0][id].append(simul*0.4)
  elif(event >= 1660352400 and event < 1660381200):
    hot_title[0][id].append(simul*0.3)
  elif(event >= 1659747600 and event < 1659776400):
    hot_title[0][id].append(simul*0.2)
  elif(event >= 1659142800 and event < 1659171600):
    hot_title[0][id].append(simul*0.1)

  if(event >= 1660986000 and event < 1661014800):
    hot_title[1][id].append(simul*0.4)
  elif(event >= 1660381200 and event < 1660410000):
    hot_title[1][id].append(simul*0.3)
  elif(event >= 1659776400 and event < 1659805200):
    hot_title[1][id].append(simul*0.2)
  elif(event >= 1659171600 and event < 1659200400):
    hot_title[1][id].append(simul*0.1)

  if(event >= 1661014800 and event < 1661029200):
    hot_title[2][id].append(simul*0.4)
  elif(event >= 1660410000 and event < 1660424400):
    hot_title[2][id].append(simul*0.3)
  elif(event >= 1659805200 and event < 1659819600):
    hot_title[2][id].append(simul*0.2)
  elif(event >= 1659200400 and event < 1659214800):
    hot_title[2][id].append(simul*0.1)

  if(event >= 1661029200 and event < 1661043600):
    hot_title[3][id].append(simul*0.4)
  elif(event >= 1660424400 and event < 1660438800):
    hot_title[3][id].append(simul*0.3)
  elif(event >= 1659819600 and event < 1659834000):
    hot_title[3][id].append(simul*0.2)
  elif(event >= 1659214800 and event < 1659229200):
    hot_title[3][id].append(simul*0.1)

  if(event >= 1661043600 and event < 1661072400):
    hot_title[4][id].append(simul*0.4)
  elif(event >= 1660438800 and event < 1660467600):
    hot_title[4][id].append(simul*0.3)
  elif(event >= 1659834000 and event < 1659862800):
    hot_title[4][id].append(simul*0.2)
  elif(event >= 1659229200 and event < 1659258000):
    hot_title[4][id].append(simul*0.1)

  if(event >= 1661072400 and event < 1661101200):
    hot_title[5][id].append(simul*0.4)
  elif(event >= 1660467600 and event < 1660496400):
    hot_title[5][id].append(simul*0.3)
  elif(event >= 1659862800 and event < 1659891600):
    hot_title[5][id].append(simul*0.2)
  elif(event >= 1659258000 and event < 1659286800):
    hot_title[5][id].append(simul*0.1)

  if(event >= 1661101200 and event < 1661115600):
    hot_title[6][id].append(simul*0.4)
  elif(event >= 1660496400 and event < 1660510800):
    hot_title[6][id].append(simul*0.3)
  elif(event >= 1659891600 and event < 1659906000):
    hot_title[6][id].append(simul*0.2)
  elif(event >= 1659286800 and event < 1659301200):
    hot_title[6][id].append(simul*0.1)

  if(event >= 1661115600 and event < 1661130000):
    hot_title[7][id].append(simul*0.4)
  elif(event >= 1660510800 and event < 1660525200):
    hot_title[7][id].append(simul*0.3)
  elif(event >= 1659906000 and event < 1659920400):
    hot_title[7][id].append(simul*0.2)
  elif(event >= 1659301200 and event < 1659315600):
    hot_title[7][id].append(simul*0.1)

  if(event >= 1661130000 and event < 1661158800):
    hot_title[8][id].append(simul*0.4)
  elif(event >= 1660525200 and event < 1660554000):
    hot_title[8][id].append(simul*0.3)
  elif(event >= 1659920400 and event < 1659949200):
    hot_title[8][id].append(simul*0.2)
  elif(event >= 1659315600 and event < 1659344400):
    hot_title[8][id].append(simul*0.1)

  if(event >= 1661158800 and event < 1661187600):
    hot_title[9][id].append(simul*0.4)
  elif(event >= 1660554000 and event < 1660582800):
    hot_title[9][id].append(simul*0.3)
  elif(event >= 1659949200 and event < 1659978000):
    hot_title[9][id].append(simul*0.2)
  elif(event >= 1659344400 and event < 1659373200):
    hot_title[9][id].append(simul*0.1)

  if(event >= 1661187600 and event < 1661202000):
    hot_title[10][id].append(simul*0.4)
  elif(event >= 1660582800 and event < 1660597200):
    hot_title[10][id].append(simul*0.3)
  elif(event >= 1659978000 and event < 1659992400):
    hot_title[10][id].append(simul*0.2)
  elif(event >= 1659373200 and event < 1659387600):
    hot_title[10][id].append(simul*0.1)

  if(event >= 1661202000 and event < 1661216400):
    hot_title[11][id].append(simul*0.4)
  elif(event >= 1660597200 and event < 1660611600):
    hot_title[11][id].append(simul*0.3)
  elif(event >= 1659992400 and event < 1660006800):
    hot_title[11][id].append(simul*0.2)
  elif(event >= 1659387600 and event < 1659402000):
    hot_title[11][id].append(simul*0.1)

  if(event >= 1661216400 and event < 1661245200):
    hot_title[12][id].append(simul*0.4)
  elif(event >= 1660611600 and event < 1660640400):
    hot_title[12][id].append(simul*0.3)
  elif(event >= 1660006800 and event < 1660035600):
    hot_title[12][id].append(simul*0.2)
  elif(event >= 1659402000 and event < 1659430800):
    hot_title[12][id].append(simul*0.1)

  if(event >= 1661245200 and event < 1661274000):
    hot_title[13][id].append(simul*0.4)
  elif(event >= 1660640400 and event < 1660669200):
    hot_title[13][id].append(simul*0.3)
  elif(event >= 1660035600 and event < 1660064400):
    hot_title[13][id].append(simul*0.2)
  elif(event >= 1659430800 and event < 1659459600):
    hot_title[13][id].append(simul*0.1)

  if(event >= 1661274000 and event < 1661288400):
    hot_title[14][id].append(simul*0.4)
  elif(event >= 1660669200 and event < 1660683600):
    hot_title[14][id].append(simul*0.3)
  elif(event >= 1660064400 and event < 1660078800):
    hot_title[14][id].append(simul*0.2)
  elif(event >= 1659459600 and event < 1659474000):
    hot_title[14][id].append(simul*0.1)

  if(event >= 1661288400 and event < 1661302800):
    hot_title[15][id].append(simul*0.4)
  elif(event >= 1660683600 and event < 1660698000):
    hot_title[15][id].append(simul*0.3)
  elif(event >= 1660078800 and event < 1660093200):
    hot_title[15][id].append(simul*0.2)
  elif(event >= 1659474000 and event < 1659488400):
    hot_title[15][id].append(simul*0.1)

  if(event >= 1661302800 and event < 1661331600):
    hot_title[16][id].append(simul*0.4)
  elif(event >= 1660698000 and event < 1660726800):
    hot_title[16][id].append(simul*0.3)
  elif(event >= 1660093200 and event < 1660122000):
    hot_title[16][id].append(simul*0.2)
  elif(event >= 1659488400 and event < 1659517200):
    hot_title[16][id].append(simul*0.1)

  if(event >= 1661331600 and event < 1661360400):
    hot_title[17][id].append(simul*0.4)
  elif(event >= 1660726800 and event < 1660755600):
    hot_title[17][id].append(simul*0.3)
  elif(event >= 1660122000 and event < 1660150800):
    hot_title[17][id].append(simul*0.2)
  elif(event >= 1659517200 and event < 1659546000):
    hot_title[17][id].append(simul*0.1)

  if(event >= 1661360400 and event < 1661374800):
    hot_title[18][id].append(simul*0.4)
  elif(event >= 1660755600 and event < 1660770000):
    hot_title[18][id].append(simul*0.3)
  elif(event >= 1660150800 and event < 1660165200):
    hot_title[18][id].append(simul*0.2)
  elif(event >= 1659546000 and event < 1659560400):
    hot_title[18][id].append(simul*0.1)

  if(event >= 1661374800 and event < 1661389200):
    hot_title[19][id].append(simul*0.4)
  elif(event >= 1660770000 and event < 1660784400):
    hot_title[19][id].append(simul*0.3)
  elif(event >= 1660165200 and event < 1660179600):
    hot_title[19][id].append(simul*0.2)
  elif(event >= 1659560400 and event < 1659574800):
    hot_title[19][id].append(simul*0.1)

  if(event >= 1661389200 and event < 1661418000):
    hot_title[20][id].append(simul*0.4)
  elif(event >= 1660784400 and event < 1660813200):
    hot_title[20][id].append(simul*0.3)
  elif(event >= 1660179600 and event < 1660208400):
    hot_title[20][id].append(simul*0.2)
  elif(event >= 1659574800 and event < 1659603600):
    hot_title[20][id].append(simul*0.1)

  if(event >= 1661418000 and event < 1661446800):
    hot_title[21][id].append(simul*0.4)
  elif(event >= 1660813200 and event < 1660842000):
    hot_title[21][id].append(simul*0.3)
  elif(event >= 1660208400 and event < 1660237200):
    hot_title[21][id].append(simul*0.2)
  elif(event >= 1659603600 and event < 1659632400):
    hot_title[21][id].append(simul*0.1)

  if(event >= 1661446800 and event < 1661461200):
    hot_title[22][id].append(simul*0.4)
  elif(event >= 1660842000 and event < 1660856400):
    hot_title[22][id].append(simul*0.3)
  elif(event >= 1660237200 and event < 1660251600):
    hot_title[22][id].append(simul*0.2)
  elif(event >= 1659632400 and event < 1659646800):
    hot_title[22][id].append(simul*0.1)

  if(event >= 1661461200 and event < 1661475600):
    hot_title[23][id].append(simul*0.4)
  elif(event >= 1660856400 and event < 1660870800):
    hot_title[23][id].append(simul*0.3)
  elif(event >= 1660251600 and event < 1660266000):
    hot_title[23][id].append(simul*0.2)
  elif(event >= 1659646800 and event < 1659661200):
    hot_title[23][id].append(simul*0.1)

  if(event >= 1661475600 and event < 1661504400):
    hot_title[24][id].append(simul*0.4)
  elif(event >= 1660870800 and event < 1660899600):
    hot_title[24][id].append(simul*0.3)
  elif(event >= 1660266000 and event < 1660294800):
    hot_title[24][id].append(simul*0.2)
  elif(event >= 1659661200 and event < 1659690000):
    hot_title[24][id].append(simul*0.1)

  if(event >= 1661504400 and event < 1661533200):
    hot_title[25][id].append(simul*0.4)
  elif(event >= 1660899600 and event < 1660928400):
    hot_title[25][id].append(simul*0.3)
  elif(event >= 1660294800 and event < 1660323600):
    hot_title[25][id].append(simul*0.2)
  elif(event >= 1659690000 and event < 1659718800):
    hot_title[25][id].append(simul*0.1)

  if(event >= 1661533200 and event < 1661547600):
    hot_title[26][id].append(simul*0.4)
  elif(event >= 1660928400 and event < 1660942800):
    hot_title[26][id].append(simul*0.3)
  elif(event >= 1660323600 and event < 1660338000):
    hot_title[26][id].append(simul*0.2)
  elif(event >= 1659718800 and event < 1659733200):
    hot_title[26][id].append(simul*0.1)

  if(event >= 1661547600 and event < 1661562000):
    hot_title[27][id].append(simul*0.4)
  elif(event >= 1660942800 and event < 1660957200):
    hot_title[27][id].append(simul*0.3)
  elif(event >= 1660338000 and event < 1660352400):
    hot_title[27][id].append(simul*0.2)
  elif(event >= 1659733200 and event < 1659747600):
    hot_title[27][id].append(simul*0.1)

In [ ]:
hot_rate = []
for i in range(28):
  for j in range(30460):
    if(len(hot_title[i][j]) == 0):
      hot_rate.append(0)
    else:
      #hot_rate.append(sum(hot_title[i][j]) / len(hot_title[i][j]))
      hot_rate.append(sum(hot_title[i][j]))

### 每小時平均觀看時間

In [ ]:
temp = train_source[train_source['event_time']>week2]

In [ ]:
temp

In [ ]:
new_temp = temp.pivot_table(index ='user_id',
               columns =['weekday','timeslot'],
               values ='played_duration',
               aggfunc = 'sum',
               fill_value = 0)

In [ ]:
new_temp.columns = ['_slot'.join(str(s).strip() for s in col if s) for col in new_temp.columns]
new_temp.reset_index(inplace=True)

In [ ]:
new_temp

,user_id,Friday,Friday_slot1,Friday_slot2,Friday_slot3,Monday,Monday_slot1,Monday_slot2,Monday_slot3,Saturday,...,Thursday_slot2,Thursday_slot3,Tuesday,Tuesday_slot1,Tuesday_slot2,Tuesday_slot3,Wednesday,Wednesday_slot1,Wednesday_slot2,Wednesday_slot3
0,0,8136,144,0,0,131568,0,0,0,0,...,0,0,118032,0,0,0,24,0,0,0
1,1,0,43824,0,62952,0,156,0,0,0,...,0,0,7362,16512,0,18126,0,9768,0,4752
2,2,67344,60006,0,0,95658,41970,0,97104,199740,...,0,0,46446,70488,0,0,32712,90552,0,0
3,3,24240,0,0,0,0,0,0,0,222552,...,0,0,0,0,0,0,0,132000,0,0
4,4,36474,144,0,0,0,0,0,0,0,...,0,0,48,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27019,30454,12408,0,0,11226,13398,0,0,4116,97800,...,0,32874,0,0,0,15312,0,0,0,12816
27020,30455,144594,7458,0,0,165024,34872,0,0,1104,...,0,0,343908,230034,0,0,0,30456,0,0
27021,30456,24972,7908,0,26538,0,0,0,0,0,...,0,5412,0,0,0,0,0,0,0,0
27022,30457,0,0,0,0,0,192,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
new_temp.loc[:,new_temp.columns != 'user_id'] /= 16

In [ ]:
new_temp[['Friday_slot3','Friday_slot2',
     'Thursday_slot3','Thursday_slot2',
     'Wednesday_slot3','Wednesday_slot2',
     'Tuesday_slot3', 'Tuesday_slot2',
     'Monday_slot3','Monday_slot2',
     'Sunday_slot3','Sunday_slot2',
     'Saturday_slot3', 'Saturday_slot2',]] *=2

In [ ]:
new_temp

,user_id,Friday,Friday_slot1,Friday_slot2,Friday_slot3,Monday,Monday_slot1,Monday_slot2,Monday_slot3,Saturday,...,Thursday_slot2,Thursday_slot3,Tuesday,Tuesday_slot1,Tuesday_slot2,Tuesday_slot3,Wednesday,Wednesday_slot1,Wednesday_slot2,Wednesday_slot3
0,0,508.500,9.000,0.0,0.00,8223.000,0.000,0.0,0.0,0.00,...,0.0,0.00,7377.000,0.000,0.0,0.00,1.5,0.000,0.0,0.0
1,1,0.000,2739.000,0.0,7869.00,0.000,9.750,0.0,0.0,0.00,...,0.0,0.00,460.125,1032.000,0.0,2265.75,0.0,610.500,0.0,594.0
2,2,4209.000,3750.375,0.0,0.00,5978.625,2623.125,0.0,12138.0,12483.75,...,0.0,0.00,2902.875,4405.500,0.0,0.00,2044.5,5659.500,0.0,0.0
3,3,1515.000,0.000,0.0,0.00,0.000,0.000,0.0,0.0,13909.50,...,0.0,0.00,0.000,0.000,0.0,0.00,0.0,8250.000,0.0,0.0
4,4,2279.625,9.000,0.0,0.00,0.000,0.000,0.0,0.0,0.00,...,0.0,0.00,3.000,0.000,0.0,0.00,0.0,0.000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27019,30454,775.500,0.000,0.0,1403.25,837.375,0.000,0.0,514.5,6112.50,...,0.0,4109.25,0.000,0.000,0.0,1914.00,0.0,0.000,0.0,1602.0
27020,30455,9037.125,466.125,0.0,0.00,10314.000,2179.500,0.0,0.0,69.00,...,0.0,0.00,21494.250,14377.125,0.0,0.00,0.0,1903.500,0.0,0.0
27021,30456,1560.750,494.250,0.0,3317.25,0.000,0.000,0.0,0.0,0.00,...,0.0,676.50,0.000,0.000,0.0,0.00,0.0,0.000,0.0,0.0
27022,30457,0.000,0.000,0.0,0.00,0.000,12.000,0.0,0.0,0.00,...,0.0,0.00,0.000,0.000,0.0,0.00,0.0,0.000,0.0,0.0


In [ ]:
test = new_temp.set_index('user_id').reindex(range(0,30460)).reset_index()
test = test.fillna(0)
print(test)

       user_id    Friday  Friday_slot1  Friday_slot2  Friday_slot3     Monday  \
0            0   508.500         9.000           0.0          0.00   8223.000   
1            1     0.000      2739.000           0.0       7869.00      0.000   
2            2  4209.000      3750.375           0.0          0.00   5978.625   
3            3  1515.000         0.000           0.0          0.00      0.000   
4            4  2279.625         9.000           0.0          0.00      0.000   
...        ...       ...           ...           ...           ...        ...   
30455    30455  9037.125       466.125           0.0          0.00  10314.000   
30456    30456  1560.750       494.250           0.0       3317.25      0.000   
30457    30457     0.000         0.000           0.0          0.00      0.000   
30458    30458     0.000      1807.125           0.0          0.00    931.500   
30459    30459     0.000         0.000           0.0          0.00      0.000   

       Monday_slot1  Monday

In [ ]:
a = []
for i in range(9,29):
  a.extend(test.iloc[:,i].values)
for i in range(1,9):
  a.extend(test.iloc[:,i].values)

In [ ]:
len(a)

852880

### 計算各用戶依星期及timeslot分類的總播放時間

In [ ]:
new_train_source = train_source.pivot_table(index ='user_id',
                         columns =['weekday','timeslot'],
                         values ='played_duration',
                         aggfunc = 'sum',
                         fill_value = 0)

In [ ]:
new_train_source.columns = ['_slot'.join(str(s).strip() for s in col if s) for col in new_train_source.columns]
new_train_source.reset_index(inplace=True)

In [ ]:
new_train_source

,user_id,Friday,Friday_slot1,Friday_slot2,Friday_slot3,Monday,Monday_slot1,Monday_slot2,Monday_slot3,Saturday,...,Thursday_slot2,Thursday_slot3,Tuesday,Tuesday_slot1,Tuesday_slot2,Tuesday_slot3,Wednesday,Wednesday_slot1,Wednesday_slot2,Wednesday_slot3
0,0,28431.5,3120.5,0.0,7.0,160594.5,12817.5,0.0,2034.5,5397.5,...,0.0,2132.0,156231.5,9670.0,617.0,1919.0,42466.0,14279.5,0.0,2132.5
1,1,6784.0,49101.0,0.0,65583.0,6408.0,6098.5,0.0,13208.5,26952.5,...,0.0,10173.0,22628.5,25445.0,0.0,34954.0,8322.0,18236.0,0.0,21977.5
2,2,96972.5,103681.5,0.0,11650.0,113080.5,52683.0,0.0,100475.0,238684.5,...,0.0,6177.0,63137.0,122188.5,0.0,1465.0,45803.5,107578.0,0.0,2865.5
3,3,30759.5,62404.5,0.0,5113.5,3442.5,57004.5,0.0,1974.5,273125.0,...,0.0,448.5,3074.0,64148.0,0.0,1788.0,11580.5,191768.0,0.0,0.0
4,4,36475.5,16779.5,5267.0,0.0,5472.0,33386.0,10295.5,635.5,28.0,...,10369.0,2253.0,52.0,6666.5,2701.5,1695.0,11583.0,11066.5,0.0,3012.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30455,30455,172247.5,76438.5,8039.0,716.5,183234.0,80253.0,3095.5,3990.5,12146.0,...,2911.5,1920.0,355323.0,269199.0,4668.5,2206.5,29578.0,83599.5,6675.5,10786.0
30456,30456,46958.5,38367.0,0.0,34340.0,14083.0,22244.0,0.0,686.0,33867.5,...,0.0,5481.0,9007.0,6299.0,0.0,708.5,12529.5,16731.0,0.0,0.0
30457,30457,6387.5,146.5,749.0,0.0,1350.5,1096.0,0.0,1311.5,311.0,...,0.0,0.0,6847.0,6688.5,4683.5,0.0,0.0,0.0,0.0,0.0
30458,30458,6378.0,47513.0,7512.5,846.5,20338.5,34858.5,7704.5,1435.0,2160.5,...,52464.0,692.5,49173.5,25197.0,27004.5,1811.0,95968.5,26543.5,392.5,0.0


依照相對日子及相對timeslot重新編排data順序，建立新的28個表格並連接

In [ ]:
column_list = ['Saturday','Friday_slot3',  'Friday_slot2','Friday_slot1', 'Friday',
       'Thursday_slot3','Thursday_slot2', 'Thursday_slot1','Thursday',
       'Wednesday_slot3','Wednesday_slot2','Wednesday_slot1','Wednesday',
       'Tuesday_slot3', 'Tuesday_slot2','Tuesday_slot1', 'Tuesday',
       'Monday_slot3','Monday_slot2', 'Monday_slot1','Monday',
       'Sunday_slot3','Sunday_slot2', 'Sunday_slot1',  'Sunday',
       'Saturday_slot3', 'Saturday_slot2','Saturday_slot1']
relative = ['day0_slot0','day0_slot1','day0_slot2','day0_slot3',
            'day1_slot0','day1_slot1','day1_slot2','day1_slot3',
            'day2_slot0','day2_slot1','day2_slot2','day2_slot3',
            'day3_slot0','day3_slot1','day3_slot2','day3_slot3',
            'day4_slot0','day4_slot1','day4_slot2','day4_slot3',
            'day5_slot0','day5_slot1','day5_slot2','day5_slot3',
            'day6_slot0','day6_slot1','day6_slot2','day6_slot3'
            ]
use_column = ['user_id']+column_list
df = new_train_source[use_column]
df.columns = ['user_id'] + relative
for i in range(0,27):
  column_list = column_list[-1:]+column_list[:-1]
  column2 = ['user_id']+column_list
  df2 = new_train_source[column2]
  df2.columns = ['user_id'] + relative
  df = df.append(df2,ignore_index=True)
train_data = df


In [ ]:
train_data['hot_rate'] = hot_rate
train_data['hot_drate'] = hot_drate

In [ ]:
train_data['avg_time'] = a

In [ ]:
train_data

,user_id,day0_slot0,day0_slot1,day0_slot2,day0_slot3,day1_slot0,day1_slot1,day1_slot2,day1_slot3,day2_slot0,...,day5_slot1,day5_slot2,day5_slot3,day6_slot0,day6_slot1,day6_slot2,day6_slot3,hot_rate,hot_drate,avg_time
0,0,5397.5,7.0,0.0,3120.5,28431.5,2132.0,0.0,27379.0,61549.5,...,677.5,0.0,5365.5,14050.5,0.0,0.0,7297.0,0.0,0.0,0.00
1,1,26952.5,65583.0,0.0,49101.0,6784.0,10173.0,0.0,6078.0,9830.5,...,905.5,0.0,3364.5,99885.0,0.0,0.0,18448.5,0.2,0.0,0.00
2,2,238684.5,11650.0,0.0,103681.5,96972.5,6177.0,0.0,33050.0,28249.0,...,93027.0,0.0,409141.5,170998.5,146759.0,0.0,231654.5,0.3,0.0,12483.75
3,3,273125.0,5113.5,0.0,62404.5,30759.5,448.5,0.0,54437.0,4410.0,...,0.0,0.0,145242.0,91776.5,877.0,0.0,60467.0,1.4,0.0,13909.50
4,4,28.0,0.0,5267.0,16779.5,36475.5,2253.0,10369.0,8964.0,6802.5,...,2051.5,8164.0,14484.0,0.0,507.5,2431.0,20636.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852875,30455,716.5,8039.0,76438.5,172247.5,1920.0,2911.5,150926.5,90966.0,10786.0,...,7367.5,327634.5,179773.5,2757.0,5938.5,97690.0,12146.0,0.0,0.0,0.00
852876,30456,34340.0,0.0,38367.0,46958.5,5481.0,0.0,33350.5,11368.0,0.0,...,0.0,8327.0,57888.0,1098.5,0.0,16400.5,33867.5,0.5,0.0,0.00
852877,30457,0.0,749.0,146.5,6387.5,0.0,0.0,0.5,0.0,0.0,...,3014.5,20977.0,2580.5,0.0,65.5,1409.5,311.0,0.0,0.0,0.00
852878,30458,846.5,7512.5,47513.0,6378.0,692.5,52464.0,160758.5,7076.0,0.0,...,4254.0,83257.0,40367.0,0.0,5306.0,20263.5,2160.5,0.0,0.0,0.00


### train_target 整理

In [ ]:
#train_target = pd.read_csv('train_target_events.csv')

In [ ]:
'''
def target_time(date):
  week_list = [2,3,4,5,6,0,1]
  struct_time = time.localtime(date)
  w_day = struct_time.tm_wday
  hour = struct_time.tm_hour
  if hour < 1:
    w_day = w_day - 1 if w_day !=0 else 6
    time_slot = 3
  elif hour < 9 :
    time_slot = 0
  elif hour < 17:
    time_slot = 1
  elif hour < 21:
    time_slot = 2
  else:
    time_slot = 3
  w_day = week_list[w_day]
  time_slot = w_day*4 + time_slot

  return time_slot
'''


'\ndef target_time(date):\n  week_list = [2,3,4,5,6,0,1]\n  struct_time = time.localtime(date)\n  w_day = struct_time.tm_wday\n  hour = struct_time.tm_hour\n  if hour < 1:\n    w_day = w_day - 1 if w_day !=0 else 6\n    time_slot = 3\n  elif hour < 9 :\n    time_slot = 0\n  elif hour < 17:\n    time_slot = 1\n  elif hour < 21:\n    time_slot = 2\n  else:\n    time_slot = 3\n  w_day = week_list[w_day]\n  time_slot = w_day*4 + time_slot\n\n  return time_slot\n'

In [ ]:
#train_target['timeslot'] = train_target['event_time'].apply(target_time)
#train_target[train_target['user_id']==1]

In [ ]:
target_labels = pd.read_csv('light_train_target_labels.csv')
df = target_labels.set_index(['user_id'])

new_target_labels = pd.melt(df, value_vars=df.columns)
new_target_labels


,variable,value
0,time_slot_0,1.0
1,time_slot_0,1.0
2,time_slot_0,0.0
3,time_slot_0,1.0
4,time_slot_0,0.0
...,...,...
852875,time_slot_27,0.0
852876,time_slot_27,0.0
852877,time_slot_27,0.0
852878,time_slot_27,0.0


合併成完整train data

In [ ]:
new_target_labels = new_target_labels.drop('variable',axis =1)
train_data['value'] = new_target_labels['value']
train_data

,user_id,day0_slot0,day0_slot1,day0_slot2,day0_slot3,day1_slot0,day1_slot1,day1_slot2,day1_slot3,day2_slot0,...,day5_slot2,day5_slot3,day6_slot0,day6_slot1,day6_slot2,day6_slot3,hot_rate,hot_drate,avg_time,value
0,0,5397.5,7.0,0.0,3120.5,28431.5,2132.0,0.0,27379.0,61549.5,...,0.0,5365.5,14050.5,0.0,0.0,7297.0,0.0,0.0,0.00,1.0
1,1,26952.5,65583.0,0.0,49101.0,6784.0,10173.0,0.0,6078.0,9830.5,...,0.0,3364.5,99885.0,0.0,0.0,18448.5,0.2,0.0,0.00,1.0
2,2,238684.5,11650.0,0.0,103681.5,96972.5,6177.0,0.0,33050.0,28249.0,...,0.0,409141.5,170998.5,146759.0,0.0,231654.5,0.3,0.0,12483.75,0.0
3,3,273125.0,5113.5,0.0,62404.5,30759.5,448.5,0.0,54437.0,4410.0,...,0.0,145242.0,91776.5,877.0,0.0,60467.0,1.4,0.0,13909.50,1.0
4,4,28.0,0.0,5267.0,16779.5,36475.5,2253.0,10369.0,8964.0,6802.5,...,8164.0,14484.0,0.0,507.5,2431.0,20636.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852875,30455,716.5,8039.0,76438.5,172247.5,1920.0,2911.5,150926.5,90966.0,10786.0,...,327634.5,179773.5,2757.0,5938.5,97690.0,12146.0,0.0,0.0,0.00,0.0
852876,30456,34340.0,0.0,38367.0,46958.5,5481.0,0.0,33350.5,11368.0,0.0,...,8327.0,57888.0,1098.5,0.0,16400.5,33867.5,0.5,0.0,0.00,0.0
852877,30457,0.0,749.0,146.5,6387.5,0.0,0.0,0.5,0.0,0.0,...,20977.0,2580.5,0.0,65.5,1409.5,311.0,0.0,0.0,0.00,0.0
852878,30458,846.5,7512.5,47513.0,6378.0,692.5,52464.0,160758.5,7076.0,0.0,...,83257.0,40367.0,0.0,5306.0,20263.5,2160.5,0.0,0.0,0.00,0.0


### 整理test data

In [ ]:
test_source = pd.read_csv('test_source_events.csv')
test_source

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,title_in_simulcast,internet_connection_type
0,30460,139323,4199117,191,1.639302e+09,39,0,0,0,6
1,30460,139323,4199117,64,1.639303e+09,1987,0,0,0,6
2,30460,139288,4201834,93,1.639488e+09,2747,1,0,1,1
3,30460,139288,4203396,178,1.639571e+09,15,1,0,1,1
4,30460,139288,4203396,178,1.639571e+09,30,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...
2390066,38075,174918,5249768,2993,1.661515e+09,60,1,0,1,1
2390067,38075,174918,5249768,2993,1.661515e+09,30,1,0,1,1
2390068,38075,174918,5249768,2993,1.661515e+09,60,1,0,1,1
2390069,38075,174918,5249768,2993,1.661515e+09,25,1,0,1,1


In [ ]:
test_source['weekday'] = test_source['event_time'].apply(week)
test_source['timeslot'] = test_source['event_time'].apply(timeslot)

In [ ]:
test_source.loc[test_source['event_time'] > week1  , 'played_duration'] *= 4
test_source.loc[test_source['event_time'] > week2, 'played_duration'] *= 3
test_source.loc[test_source['event_time'] > week3, 'played_duration'] *= 2
test_source.loc[test_source['event_time'] < week3, 'played_duration'] *= 0.5

每小時觀看

In [ ]:
temp = test_source[test_source['event_time']>week2]

In [ ]:
new_temp = temp.pivot_table(index ='user_id',
               columns =['weekday','timeslot'],
               values ='played_duration',
               aggfunc = 'sum',
               fill_value = 0)

In [ ]:
new_temp.columns = ['_slot'.join(str(s).strip() for s in col if s) for col in new_temp.columns]
new_temp.reset_index(inplace=True)

In [ ]:
new_temp.loc[:,new_temp.columns != 'user_id'] /= 16

In [ ]:
new_temp[['Friday_slot3','Friday_slot2',
     'Thursday_slot3','Thursday_slot2',
     'Wednesday_slot3','Wednesday_slot2',
     'Tuesday_slot3', 'Tuesday_slot2',
     'Monday_slot3','Monday_slot2',
     'Sunday_slot3','Sunday_slot2',
     'Saturday_slot3', 'Saturday_slot2',]] *=2

In [ ]:
test = new_temp.set_index('user_id').reindex(range(30460,38076)).reset_index()
test = test.fillna(0)
print(test)

      user_id    Friday  Friday_slot1  Friday_slot2  Friday_slot3  Monday  \
0       30460     0.000         0.000           0.0           0.0     0.0   
1       30461     0.000        20.625           0.0           0.0     0.0   
2       30462     0.000         0.000           0.0           0.0     0.0   
3       30463     0.000      8233.500           0.0           0.0     0.0   
4       30464   293.625         0.000           0.0           0.0     0.0   
...       ...       ...           ...           ...           ...     ...   
7611    38071     0.000         0.000           0.0           0.0     0.0   
7612    38072     0.000         0.000           0.0           0.0     0.0   
7613    38073  5136.000         0.000           0.0           0.0     0.0   
7614    38074  1960.500       625.500        1239.0           0.0     0.0   
7615    38075     0.000       738.000           0.0           0.0     0.0   

      Monday_slot1  Monday_slot2  Monday_slot3  Saturday  ...  Thursday_slo

In [ ]:
a = []
for i in range(9,29):
  a.extend(test.iloc[:,i].values)
for i in range(1,9):
  a.extend(test.iloc[:,i].values)

In [ ]:
len(a)

213248

In [ ]:
new_test_source = test_source.pivot_table(index ='user_id',
                         columns =['weekday','timeslot'],
                         values ='played_duration',
                         aggfunc = 'sum',
                         fill_value = 0)

In [ ]:
new_test_source

weekday    Friday                             Monday                     \
timeslot        0         1       2       3        0         1        2   
user_id                                                                   
30460     11401.5   19040.0   303.0  2158.5  19960.5   30879.5   7790.0   
30461      5642.0   23075.5     0.0     0.0   4512.0   12223.5      0.0   
30462      2463.0    1450.5  6688.0  1035.5   1500.0   65683.5   9027.5   
30463     10696.5  156912.5     0.0     0.0   5660.0   87037.5      0.0   
30464      8430.0   18590.5     0.0   655.0   5449.0  154520.5      0.0   
...           ...       ...     ...     ...      ...       ...      ...   
38071         0.0       0.0     0.0     0.0   2054.5    6140.0   3837.0   
38072         0.0       0.0     0.0     0.0      0.0     168.0  66072.0   
38073     82176.0       0.0     0.0     0.0      0.0       0.0      0.0   
38074     31368.0   10008.0  9912.0     0.0      0.0       0.0      0.0   
38075         0.0   11808.0     0.0     0.0      0.0       0.0      0.0   

weekday           Saturday           ... Thursday          Tuesday            \
timeslot        3        0        1  ...        2       3        0         1   
user_id                              ...                                       
30460     13114.0   4673.5  36331.5  ...   4630.5  4741.5   8551.5   16985.5   
30461       728.0  27079.0  16092.0  ...      0.0     0.0   1860.0   83666.0   
30462     13089.0   7667.0   4659.5  ...   8133.5  5684.0   1486.0   15639.5   
30463         0.0   1428.0  43269.0  ...      0.0     0.0   8496.0   22966.0   
30464      1151.5  44778.0  11587.5  ...      0.0   582.0   2830.5   56537.0   
...           ...      ...      ...  ...      ...     ...      ...       ...   
38071         0.0     45.5   2496.5  ...      0.0     0.0  11644.0    4973.0   
38072         0.0      0.0      0.0  ...      0.0     0.0      0.0  128712.0   
38073         0.0      0.0      0.0  ...      0.0     0.0      0.0   71256.0   
38074         0.0      0.0      0.0  ...      0.0     0.0      0.0       0.0   
38075         0.0      0.0      0.0  ...      0.0     0.0      0.0       0.0   

weekday                    Wednesday                             
timeslot         2       3         0         1        2       3  
user_id                                                          
30460       3726.0  1542.5   16079.5   13880.5   7798.5  6257.0  
30461          0.0     0.0    1812.0   15411.5      0.0   179.0  
30462      13837.0  7247.0    1394.0    8575.0   9139.5  3169.5  
30463          0.0     0.0    7936.0    5848.5      0.0     0.0  
30464          0.0     0.0     688.5   49316.0      0.0  3210.5  
...            ...     ...       ...       ...      ...     ...  
38071       5619.5     0.0       0.0       0.0      0.0     0.0  
38072          0.0     0.0       0.0       0.0      0.0     0.0  
38073     138000.0     0.0   49632.0  156312.0  28392.0     0.0  
38074          0.0     0.0       0.0   35952.0      0.0     0.0  
38075          0.0     0.0       0.0       0.0      0.0     0.0  

[7616 rows x 28 columns]

In [ ]:
new_test_source.columns = ['_slot'.join(str(s).strip() for s in col if s) for col in new_test_source.columns]
new_test_source.reset_index(inplace=True)

In [ ]:
column_list = ['Saturday','Friday_slot3',  'Friday_slot2','Friday_slot1', 'Friday',
       'Thursday_slot3','Thursday_slot2', 'Thursday_slot1','Thursday',
       'Wednesday_slot3','Wednesday_slot2','Wednesday_slot1','Wednesday',
       'Tuesday_slot3', 'Tuesday_slot2','Tuesday_slot1', 'Tuesday',
       'Monday_slot3','Monday_slot2', 'Monday_slot1','Monday',
       'Sunday_slot3','Sunday_slot2', 'Sunday_slot1',  'Sunday',
       'Saturday_slot3', 'Saturday_slot2','Saturday_slot1']
relative = ['day0_slot0','day0_slot1','day0_slot2','day0_slot3',
            'day1_slot0','day1_slot1','day1_slot2','day1_slot3',
            'day2_slot0','day2_slot1','day2_slot2','day2_slot3',
            'day3_slot0','day3_slot1','day3_slot2','day3_slot3',
            'day4_slot0','day4_slot1','day4_slot2','day4_slot3',
            'day5_slot0','day5_slot1','day5_slot2','day5_slot3',
            'day6_slot0','day6_slot1','day6_slot2','day6_slot3'
            ]
use_column = ['user_id']+column_list
df = new_test_source[use_column]
df.columns = ['user_id'] + relative
for i in range(0,27):
  column_list = column_list[-1:]+column_list[:-1]
  column2 = ['user_id']+column_list
  df2 = new_test_source[column2]
  df2.columns = ['user_id'] + relative
  df = df.append(df2,ignore_index=True)
test_data = df

In [ ]:
hot_drama = dict()
for i in range(28):
  hot_drama[i] = dict()
  for j in range(30460,38076):
    hot_drama[i][j] = []
for i in range(len(test_source['user_id'])):
  event = test_source['event_time'][i]
  id = test_source['user_id'][i]
  simul = test_source['title_id'][i]
  if(event < 1659056400):
    continue
  if(event >= 1660957200 and event < 1660986000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)
  elif(event >= 1660352400 and event < 1660381200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)
  elif(event >= 1659747600 and event < 1659776400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)
  elif(event >= 1659142800 and event < 1659171600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[0][id].append(1)
    else:
      hot_drama[0][id].append(0)

  if(event >= 1660986000 and event < 1661014800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)
  elif(event >= 1660381200 and event < 1660410000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)
  elif(event >= 1659776400 and event < 1659805200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)
  elif(event >= 1659171600 and event < 1659200400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[1][id].append(1)
    else:
      hot_drama[1][id].append(0)

  if(event >= 1661014800 and event < 1661029200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)
  elif(event >= 1660410000 and event < 1660424400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)
  elif(event >= 1659805200 and event < 1659819600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)
  elif(event >= 1659200400 and event < 1659214800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[2][id].append(1)
    else:
      hot_drama[2][id].append(0)

  if(event >= 1661029200 and event < 1661043600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)
  elif(event >= 1660424400 and event < 1660438800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)
  elif(event >= 1659819600 and event < 1659834000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)
  elif(event >= 1659214800 and event < 1659229200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[3][id].append(1)
    else:
      hot_drama[3][id].append(0)

  if(event >= 1661043600 and event < 1661072400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)
  elif(event >= 1660438800 and event < 1660467600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)
  elif(event >= 1659834000 and event < 1659862800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)
  elif(event >= 1659229200 and event < 1659258000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[4][id].append(1)
    else:
      hot_drama[4][id].append(0)

  if(event >= 1661072400 and event < 1661101200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)
  elif(event >= 1660467600 and event < 1660496400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)
  elif(event >= 1659862800 and event < 1659891600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)
  elif(event >= 1659258000 and event < 1659286800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[5][id].append(1)
    else:
      hot_drama[5][id].append(0)

  if(event >= 1661101200 and event < 1661115600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)
  elif(event >= 1660496400 and event < 1660510800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)
  elif(event >= 1659891600 and event < 1659906000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)
  elif(event >= 1659286800 and event < 1659301200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[6][id].append(1)
    else:
      hot_drama[6][id].append(0)

  if(event >= 1661115600 and event < 1661130000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)
  elif(event >= 1660510800 and event < 1660525200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)
  elif(event >= 1659906000 and event < 1659920400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)
  elif(event >= 1659301200 and event < 1659315600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[7][id].append(1)
    else:
      hot_drama[7][id].append(0)

  if(event >= 1661130000 and event < 1661158800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)
  elif(event >= 1660525200 and event < 1660554000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)
  elif(event >= 1659920400 and event < 1659949200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)
  elif(event >= 1659315600 and event < 1659344400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[8][id].append(1)
    else:
      hot_drama[8][id].append(0)

  if(event >= 1661158800 and event < 1661187600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)
  elif(event >= 1660554000 and event < 1660582800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)
  elif(event >= 1659949200 and event < 1659978000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)
  elif(event >= 1659344400 and event < 1659373200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[9][id].append(1)
    else:
      hot_drama[9][id].append(0)

  if(event >= 1661187600 and event < 1661202000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)
  elif(event >= 1660582800 and event < 1660597200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)
  elif(event >= 1659978000 and event < 1659992400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)
  elif(event >= 1659373200 and event < 1659387600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[10][id].append(1)
    else:
      hot_drama[10][id].append(0)

  if(event >= 1661202000 and event < 1661216400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)
  elif(event >= 1660597200 and event < 1660611600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)
  elif(event >= 1659992400 and event < 1660006800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)
  elif(event >= 1659387600 and event < 1659402000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[11][id].append(1)
    else:
      hot_drama[11][id].append(0)

  if(event >= 1661216400 and event < 1661245200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)
  elif(event >= 1660611600 and event < 1660640400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)
  elif(event >= 1660006800 and event < 1660035600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)
  elif(event >= 1659402000 and event < 1659430800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[12][id].append(1)
    else:
      hot_drama[12][id].append(0)

  if(event >= 1661245200 and event < 1661274000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)
  elif(event >= 1660640400 and event < 1660669200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)
  elif(event >= 1660035600 and event < 1660064400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)
  elif(event >= 1659430800 and event < 1659459600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[13][id].append(1)
    else:
      hot_drama[13][id].append(0)

  if(event >= 1661274000 and event < 1661288400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)
  elif(event >= 1660669200 and event < 1660683600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)
  elif(event >= 1660064400 and event < 1660078800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)
  elif(event >= 1659459600 and event < 1659474000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[14][id].append(1)
    else:
      hot_drama[14][id].append(0)

  if(event >= 1661288400 and event < 1661302800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)
  elif(event >= 1660683600 and event < 1660698000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)
  elif(event >= 1660078800 and event < 1660093200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)
  elif(event >= 1659474000 and event < 1659488400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[15][id].append(1)
    else:
      hot_drama[15][id].append(0)

  if(event >= 1661302800 and event < 1661331600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)
  elif(event >= 1660698000 and event < 1660726800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)
  elif(event >= 1660093200 and event < 1660122000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)
  elif(event >= 1659488400 and event < 1659517200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[16][id].append(1)
    else:
      hot_drama[16][id].append(0)

  if(event >= 1661331600 and event < 1661360400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)
  elif(event >= 1660726800 and event < 1660755600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)
  elif(event >= 1660122000 and event < 1660150800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)
  elif(event >= 1659517200 and event < 1659546000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[17][id].append(1)
    else:
      hot_drama[17][id].append(0)

  if(event >= 1661360400 and event < 1661374800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)
  elif(event >= 1660755600 and event < 1660770000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)
  elif(event >= 1660150800 and event < 1660165200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)
  elif(event >= 1659546000 and event < 1659560400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[18][id].append(1)
    else:
      hot_drama[18][id].append(0)

  if(event >= 1661374800 and event < 1661389200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)
  elif(event >= 1660770000 and event < 1660784400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)
  elif(event >= 1660165200 and event < 1660179600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)
  elif(event >= 1659560400 and event < 1659574800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[19][id].append(1)
    else:
      hot_drama[19][id].append(0)

  if(event >= 1661389200 and event < 1661418000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)
  elif(event >= 1660784400 and event < 1660813200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)
  elif(event >= 1660179600 and event < 1660208400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)
  elif(event >= 1659574800 and event < 1659603600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[20][id].append(1)
    else:
      hot_drama[20][id].append(0)

  if(event >= 1661418000 and event < 1661446800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)
  elif(event >= 1660813200 and event < 1660842000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)
  elif(event >= 1660208400 and event < 1660237200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)
  elif(event >= 1659603600 and event < 1659632400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[21][id].append(1)
    else:
      hot_drama[21][id].append(0)

  if(event >= 1661446800 and event < 1661461200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)
  elif(event >= 1660842000 and event < 1660856400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)
  elif(event >= 1660237200 and event < 1660251600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)
  elif(event >= 1659632400 and event < 1659646800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[22][id].append(1)
    else:
      hot_drama[22][id].append(0)

  if(event >= 1661461200 and event < 1661475600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)
  elif(event >= 1660856400 and event < 1660870800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)
  elif(event >= 1660251600 and event < 1660266000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)
  elif(event >= 1659646800 and event < 1659661200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[23][id].append(1)
    else:
      hot_drama[23][id].append(0)

  if(event >= 1661475600 and event < 1661504400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)
  elif(event >= 1660870800 and event < 1660899600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)
  elif(event >= 1660266000 and event < 1660294800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)
  elif(event >= 1659661200 and event < 1659690000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[24][id].append(1)
    else:
      hot_drama[24][id].append(0)

  if(event >= 1661504400 and event < 1661533200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)
  elif(event >= 1660899600 and event < 1660928400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)
  elif(event >= 1660294800 and event < 1660323600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)
  elif(event >= 1659690000 and event < 1659718800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[25][id].append(1)
    else:
      hot_drama[25][id].append(0)

  if(event >= 1661533200 and event < 1661547600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)
  elif(event >= 1660928400 and event < 1660942800):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)
  elif(event >= 1660323600 and event < 1660338000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)
  elif(event >= 1659718800 and event < 1659733200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[26][id].append(1)
    else:
      hot_drama[26][id].append(0)

  if(event >= 1661547600 and event < 1661562000):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)
  elif(event >= 1660942800 and event < 1660957200):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)
  elif(event >= 1660338000 and event < 1660352400):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)
  elif(event >= 1659733200 and event < 1659747600):
    if(simul == 68 or simul == 14 or simul == 93):
      hot_drama[27][id].append(1)
    else:
      hot_drama[27][id].append(0)

In [ ]:
hot_drate = []
for i in range(28):
  for j in range(30460,38076):
    if(len(hot_drama[i][j]) == 0):
      hot_drate.append(0)
    else:
      hot_drate.append(sum(hot_drama[i][j]) / len(hot_drama[i][j]))
      #hot_rate.append(sum(hot_title[i][j]))

In [ ]:
hot_title = dict()
for i in range(28):
  hot_title[i] = dict()
  for j in range(30460,38076):
    hot_title[i][j] = []
for i in range(len(test_source['user_id'])):
  event = test_source['event_time'][i]
  id = test_source['user_id'][i]
  simul = test_source['title_in_simulcast'][i]

  if(event >= 1660957200 and event < 1660986000):
    hot_title[0][id].append(simul*0.4)
  elif(event >= 1660352400 and event < 1660381200):
    hot_title[0][id].append(simul*0.3)
  elif(event >= 1659747600 and event < 1659776400):
    hot_title[0][id].append(simul*0.2)
  elif(event >= 1659142800 and event < 1659171600):
    hot_title[0][id].append(simul*0.1)

  if(event >= 1660986000 and event < 1661014800):
    hot_title[1][id].append(simul*0.4)
  elif(event >= 1660381200 and event < 1660410000):
    hot_title[1][id].append(simul*0.3)
  elif(event >= 1659776400 and event < 1659805200):
    hot_title[1][id].append(simul*0.2)
  elif(event >= 1659171600 and event < 1659200400):
    hot_title[1][id].append(simul*0.1)

  if(event >= 1661014800 and event < 1661029200):
    hot_title[2][id].append(simul*0.4)
  elif(event >= 1660410000 and event < 1660424400):
    hot_title[2][id].append(simul*0.3)
  elif(event >= 1659805200 and event < 1659819600):
    hot_title[2][id].append(simul*0.2)
  elif(event >= 1659200400 and event < 1659214800):
    hot_title[2][id].append(simul*0.1)

  if(event >= 1661029200 and event < 1661043600):
    hot_title[3][id].append(simul*0.4)
  elif(event >= 1660424400 and event < 1660438800):
    hot_title[3][id].append(simul*0.3)
  elif(event >= 1659819600 and event < 1659834000):
    hot_title[3][id].append(simul*0.2)
  elif(event >= 1659214800 and event < 1659229200):
    hot_title[3][id].append(simul*0.1)

  if(event >= 1661043600 and event < 1661072400):
    hot_title[4][id].append(simul*0.4)
  elif(event >= 1660438800 and event < 1660467600):
    hot_title[4][id].append(simul*0.3)
  elif(event >= 1659834000 and event < 1659862800):
    hot_title[4][id].append(simul*0.2)
  elif(event >= 1659229200 and event < 1659258000):
    hot_title[4][id].append(simul*0.1)

  if(event >= 1661072400 and event < 1661101200):
    hot_title[5][id].append(simul*0.4)
  elif(event >= 1660467600 and event < 1660496400):
    hot_title[5][id].append(simul*0.3)
  elif(event >= 1659862800 and event < 1659891600):
    hot_title[5][id].append(simul*0.2)
  elif(event >= 1659258000 and event < 1659286800):
    hot_title[5][id].append(simul*0.1)

  if(event >= 1661101200 and event < 1661115600):
    hot_title[6][id].append(simul*0.4)
  elif(event >= 1660496400 and event < 1660510800):
    hot_title[6][id].append(simul*0.3)
  elif(event >= 1659891600 and event < 1659906000):
    hot_title[6][id].append(simul*0.2)
  elif(event >= 1659286800 and event < 1659301200):
    hot_title[6][id].append(simul*0.1)

  if(event >= 1661115600 and event < 1661130000):
    hot_title[7][id].append(simul*0.4)
  elif(event >= 1660510800 and event < 1660525200):
    hot_title[7][id].append(simul*0.3)
  elif(event >= 1659906000 and event < 1659920400):
    hot_title[7][id].append(simul*0.2)
  elif(event >= 1659301200 and event < 1659315600):
    hot_title[7][id].append(simul*0.1)

  if(event >= 1661130000 and event < 1661158800):
    hot_title[8][id].append(simul*0.4)
  elif(event >= 1660525200 and event < 1660554000):
    hot_title[8][id].append(simul*0.3)
  elif(event >= 1659920400 and event < 1659949200):
    hot_title[8][id].append(simul*0.2)
  elif(event >= 1659315600 and event < 1659344400):
    hot_title[8][id].append(simul*0.1)

  if(event >= 1661158800 and event < 1661187600):
    hot_title[9][id].append(simul*0.4)
  elif(event >= 1660554000 and event < 1660582800):
    hot_title[9][id].append(simul*0.3)
  elif(event >= 1659949200 and event < 1659978000):
    hot_title[9][id].append(simul*0.2)
  elif(event >= 1659344400 and event < 1659373200):
    hot_title[9][id].append(simul*0.1)

  if(event >= 1661187600 and event < 1661202000):
    hot_title[10][id].append(simul*0.4)
  elif(event >= 1660582800 and event < 1660597200):
    hot_title[10][id].append(simul*0.3)
  elif(event >= 1659978000 and event < 1659992400):
    hot_title[10][id].append(simul*0.2)
  elif(event >= 1659373200 and event < 1659387600):
    hot_title[10][id].append(simul*0.1)

  if(event >= 1661202000 and event < 1661216400):
    hot_title[11][id].append(simul*0.4)
  elif(event >= 1660597200 and event < 1660611600):
    hot_title[11][id].append(simul*0.3)
  elif(event >= 1659992400 and event < 1660006800):
    hot_title[11][id].append(simul*0.2)
  elif(event >= 1659387600 and event < 1659402000):
    hot_title[11][id].append(simul*0.1)

  if(event >= 1661216400 and event < 1661245200):
    hot_title[12][id].append(simul*0.4)
  elif(event >= 1660611600 and event < 1660640400):
    hot_title[12][id].append(simul*0.3)
  elif(event >= 1660006800 and event < 1660035600):
    hot_title[12][id].append(simul*0.2)
  elif(event >= 1659402000 and event < 1659430800):
    hot_title[12][id].append(simul*0.1)

  if(event >= 1661245200 and event < 1661274000):
    hot_title[13][id].append(simul*0.4)
  elif(event >= 1660640400 and event < 1660669200):
    hot_title[13][id].append(simul*0.3)
  elif(event >= 1660035600 and event < 1660064400):
    hot_title[13][id].append(simul*0.2)
  elif(event >= 1659430800 and event < 1659459600):
    hot_title[13][id].append(simul*0.1)

  if(event >= 1661274000 and event < 1661288400):
    hot_title[14][id].append(simul*0.4)
  elif(event >= 1660669200 and event < 1660683600):
    hot_title[14][id].append(simul*0.3)
  elif(event >= 1660064400 and event < 1660078800):
    hot_title[14][id].append(simul*0.2)
  elif(event >= 1659459600 and event < 1659474000):
    hot_title[14][id].append(simul*0.1)

  if(event >= 1661288400 and event < 1661302800):
    hot_title[15][id].append(simul*0.4)
  elif(event >= 1660683600 and event < 1660698000):
    hot_title[15][id].append(simul*0.3)
  elif(event >= 1660078800 and event < 1660093200):
    hot_title[15][id].append(simul*0.2)
  elif(event >= 1659474000 and event < 1659488400):
    hot_title[15][id].append(simul*0.1)

  if(event >= 1661302800 and event < 1661331600):
    hot_title[16][id].append(simul*0.4)
  elif(event >= 1660698000 and event < 1660726800):
    hot_title[16][id].append(simul*0.3)
  elif(event >= 1660093200 and event < 1660122000):
    hot_title[16][id].append(simul*0.2)
  elif(event >= 1659488400 and event < 1659517200):
    hot_title[16][id].append(simul*0.1)

  if(event >= 1661331600 and event < 1661360400):
    hot_title[17][id].append(simul*0.4)
  elif(event >= 1660726800 and event < 1660755600):
    hot_title[17][id].append(simul*0.3)
  elif(event >= 1660122000 and event < 1660150800):
    hot_title[17][id].append(simul*0.2)
  elif(event >= 1659517200 and event < 1659546000):
    hot_title[17][id].append(simul*0.1)

  if(event >= 1661360400 and event < 1661374800):
    hot_title[18][id].append(simul*0.4)
  elif(event >= 1660755600 and event < 1660770000):
    hot_title[18][id].append(simul*0.3)
  elif(event >= 1660150800 and event < 1660165200):
    hot_title[18][id].append(simul*0.2)
  elif(event >= 1659546000 and event < 1659560400):
    hot_title[18][id].append(simul*0.1)

  if(event >= 1661374800 and event < 1661389200):
    hot_title[19][id].append(simul*0.4)
  elif(event >= 1660770000 and event < 1660784400):
    hot_title[19][id].append(simul*0.3)
  elif(event >= 1660165200 and event < 1660179600):
    hot_title[19][id].append(simul*0.2)
  elif(event >= 1659560400 and event < 1659574800):
    hot_title[19][id].append(simul*0.1)

  if(event >= 1661389200 and event < 1661418000):
    hot_title[20][id].append(simul*0.4)
  elif(event >= 1660784400 and event < 1660813200):
    hot_title[20][id].append(simul*0.3)
  elif(event >= 1660179600 and event < 1660208400):
    hot_title[20][id].append(simul*0.2)
  elif(event >= 1659574800 and event < 1659603600):
    hot_title[20][id].append(simul*0.1)

  if(event >= 1661418000 and event < 1661446800):
    hot_title[21][id].append(simul*0.4)
  elif(event >= 1660813200 and event < 1660842000):
    hot_title[21][id].append(simul*0.3)
  elif(event >= 1660208400 and event < 1660237200):
    hot_title[21][id].append(simul*0.2)
  elif(event >= 1659603600 and event < 1659632400):
    hot_title[21][id].append(simul*0.1)

  if(event >= 1661446800 and event < 1661461200):
    hot_title[22][id].append(simul*0.4)
  elif(event >= 1660842000 and event < 1660856400):
    hot_title[22][id].append(simul*0.3)
  elif(event >= 1660237200 and event < 1660251600):
    hot_title[22][id].append(simul*0.2)
  elif(event >= 1659632400 and event < 1659646800):
    hot_title[22][id].append(simul*0.1)

  if(event >= 1661461200 and event < 1661475600):
    hot_title[23][id].append(simul*0.4)
  elif(event >= 1660856400 and event < 1660870800):
    hot_title[23][id].append(simul*0.3)
  elif(event >= 1660251600 and event < 1660266000):
    hot_title[23][id].append(simul*0.2)
  elif(event >= 1659646800 and event < 1659661200):
    hot_title[23][id].append(simul*0.1)

  if(event >= 1661475600 and event < 1661504400):
    hot_title[24][id].append(simul*0.4)
  elif(event >= 1660870800 and event < 1660899600):
    hot_title[24][id].append(simul*0.3)
  elif(event >= 1660266000 and event < 1660294800):
    hot_title[24][id].append(simul*0.2)
  elif(event >= 1659661200 and event < 1659690000):
    hot_title[24][id].append(simul*0.1)

  if(event >= 1661504400 and event < 1661533200):
    hot_title[25][id].append(simul*0.4)
  elif(event >= 1660899600 and event < 1660928400):
    hot_title[25][id].append(simul*0.3)
  elif(event >= 1660294800 and event < 1660323600):
    hot_title[25][id].append(simul*0.2)
  elif(event >= 1659690000 and event < 1659718800):
    hot_title[25][id].append(simul*0.1)

  if(event >= 1661533200 and event < 1661547600):
    hot_title[26][id].append(simul*0.4)
  elif(event >= 1660928400 and event < 1660942800):
    hot_title[26][id].append(simul*0.3)
  elif(event >= 1660323600 and event < 1660338000):
    hot_title[26][id].append(simul*0.2)
  elif(event >= 1659718800 and event < 1659733200):
    hot_title[26][id].append(simul*0.1)

  if(event >= 1661547600 and event < 1661562000):
    hot_title[27][id].append(simul*0.4)
  elif(event >= 1660942800 and event < 1660957200):
    hot_title[27][id].append(simul*0.3)
  elif(event >= 1660338000 and event < 1660352400):
    hot_title[27][id].append(simul*0.2)
  elif(event >= 1659733200 and event < 1659747600):
    hot_title[27][id].append(simul*0.1)

In [ ]:
hot_rate = []
for i in range(28):
  for j in range(30460,38076):
    if(len(hot_title[i][j]) == 0):
      hot_rate.append(0)
    else:
      #hot_rate.append(sum(hot_title[i][j]) / len(hot_title[i][j]))
      hot_rate.append(sum(hot_title[i][j]))

In [ ]:
test_data['hot_rate'] = hot_rate
test_data['hot_drate'] = hot_drate

In [ ]:
test_data['avg_time'] = a

In [ ]:
test_data

### model

In [ ]:
from catboost import CatBoostRegressor
params = {
    'iterations':600,
    'learning_rate':0.01,
    'depth':10,
    'random_seed' : 23,
    'bagging_temperature' : 0.2,
    'od_type':'Iter',
    'metric_period' : 75,
    'od_wait':100
}

X_train = train_data.drop('value',axis=1)
y_train = train_data['value']

cat = CatBoostRegressor(**params)

In [ ]:
cat.fit(X_train, y_train)
prediction = cat.predict(test_data)

0:	learn: 0.3458075	total: 745ms	remaining: 7m 26s
75:	learn: 0.3212891	total: 1m	remaining: 6m 55s
150:	learn: 0.3145086	total: 2m	remaining: 5m 57s
225:	learn: 0.3123122	total: 2m 58s	remaining: 4m 55s
300:	learn: 0.3113209	total: 3m 58s	remaining: 3m 57s
375:	learn: 0.3106827	total: 5m 3s	remaining: 3m
450:	learn: 0.3101811	total: 6m 1s	remaining: 1m 59s
525:	learn: 0.3097416	total: 7m 2s	remaining: 59.4s
599:	learn: 0.3093251	total: 8m	remaining: 0us


In [ ]:
sample = pd.read_csv('sample.csv')
for i in range(0,28):
  sample['time_slot'+'_'+str(i)] = prediction[7616*i:7616*(i+1)]
sample[sample<0] =0
sample

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,30460,0.127315,0.321026,0.054587,0.058265,0.128408,0.224918,0.063936,0.082906,0.163586,...,0.072246,0.080311,0.171645,0.175722,0.064750,0.081159,0.155037,0.199741,0.053201,0.062062
1,30461,0.181251,0.329320,0.000000,0.014560,0.133039,0.401300,0.000000,0.015857,0.102840,...,0.000000,0.016908,0.105391,0.332316,0.000000,0.012934,0.138024,0.311724,0.000000,0.014217
2,30462,0.120806,0.121461,0.139440,0.076362,0.116120,0.184341,0.167641,0.198747,0.137102,...,0.154357,0.089399,0.163707,0.172596,0.134094,0.077754,0.096681,0.145882,0.128155,0.094292
3,30463,0.137188,0.433283,0.000502,0.005568,0.283036,0.386910,0.000059,0.005933,0.153801,...,0.000000,0.004455,0.097350,0.370188,0.000000,0.004409,0.127043,0.421874,0.000868,0.004894
4,30464,0.304194,0.325199,0.003157,0.065649,0.305157,0.456897,0.001200,0.128556,0.134677,...,0.000343,0.079737,0.139084,0.427180,0.002991,0.078631,0.193926,0.378474,0.001618,0.049439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,38071,0.065929,0.115873,0.086592,0.019398,0.084461,0.129927,0.099516,0.016378,0.099784,...,0.063672,0.013894,0.047289,0.087756,0.054765,0.014296,0.045581,0.094866,0.066462,0.016506
7612,38072,0.054628,0.198645,0.127119,0.028800,0.045901,0.201701,0.142950,0.029289,0.047113,...,0.125532,0.034793,0.047709,0.235235,0.104356,0.032543,0.061166,0.253764,0.132889,0.031478
7613,38073,0.216723,0.256094,0.086285,0.031852,0.191779,0.249727,0.113316,0.025884,0.242302,...,0.169914,0.030632,0.312559,0.207700,0.151057,0.026745,0.206352,0.268646,0.106079,0.028721
7614,38074,0.121509,0.131415,0.073877,0.027557,0.081842,0.115298,0.051500,0.026445,0.066716,...,0.050043,0.026505,0.102022,0.214735,0.107915,0.030896,0.161444,0.186208,0.118751,0.029803


In [ ]:
sample.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv',index=False)

加權前四個禮拜紀錄

In [ ]:
test = pd.read_csv('light_test_source_labels.csv')
test.head()

In [ ]:
new_test = test.drop(test.iloc[:,1:925],axis =1)
new_test.head()

In [ ]:
a = []
a.append(new_test['user_id'].values)
a

In [ ]:
for i in range(0,28):
  prob = new_test.iloc[:,i+1]*0.1 + new_test.iloc[:,i+29]*0.2 + new_test.iloc[:,i+57]*0.3 + new_test.iloc[:,i+85]*0.4
  a.append(prob.values)
a
df = pd.DataFrame(a)
df = df.T
df.columns = ['user_id',
'time_slot_0','time_slot_1','time_slot_2','time_slot_3',
'time_slot_4','time_slot_5','time_slot_6','time_slot_7',
'time_slot_8','time_slot_9','time_slot_10','time_slot_11',
'time_slot_12','time_slot_13','time_slot_14','time_slot_15',
'time_slot_16','time_slot_17','time_slot_18','time_slot_19',
'time_slot_20','time_slot_21','time_slot_22','time_slot_23',
'time_slot_24','time_slot_25','time_slot_26','time_slot_27']
df['user_id'] = df['user_id'].astype(int)
df

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv',index=False)